### Imports

In [1]:
from utils_1 import VerbParser, NounParser
import pandas as pd
import re
import os

### Loading data

In [2]:
pd.set_option('display.max_rows', None)
path = r"D:\ERC_Bang\Tasks\data_cleaning" 
os.chdir(path)
data = pd.read_csv("data.tsv", sep="\t", encoding="utf-8")

### Nouns

In [3]:
nouns = data[data["SINGULAR"].notna()]
nouns=nouns[["VALUE_ORG", "SINGULAR"]]
use=data["SINGULAR"]
use.head(10)

0       jáwdì-m
1       dá:bà-m
2      àrsɛ̌:-m
3       kɔ́mbɔ̀
4      bɛ́lɛ̀-g
5     bɛ́lɛ̀-gù
6         bɛ̀lú
7         bɛ̀lú
8    gàr sɛ̀gɛ́
9      à sɛ̀gɛ́
Name: SINGULAR, dtype: object

### Parsing nouns

In [4]:
noun_parser=NounParser()
verb_parser=VerbParser()

In [9]:
use[100:200].apply(
    lambda x: noun_parser.first_parse_durationals(x)
).apply(lambda x: noun_parser.second_parse_durationals(x)).apply(
    lambda x: noun_parser.cvcv_segmentation(x)
).apply(
    lambda x: noun_parser.nasalized_stops(x)
).apply(
lambda x: noun_parser.hyphen_space(x)
)

100        náá yɔ̀á
101       nɛ̀ɛ yɛ̌ɛ
102           nɛ̀ɛ́
103       nàŋà yǎ-á
104        nàŋà náá
105         nàà yàá
106         nàŋà-yá
107         nàà yɛ̀
108         nàà yǎà
109      nàà árⁿà-m
110       nààgù ánà
111        nàŋà àná
112       nàŋà àrⁿá
113        nàà wálá
114        nɛ̀ɛ ánè
115       nàŋà árⁿâ
116        nàà árⁿá
117       nàŋà àrⁿá
118         nàà àná
119    nàʔⁿà jù-ŋgú
120        nàŋà àrá
121        nàà àrⁿá
122           ìjɛ́ 
123         ǹjèrû-m
124         ìnjɛ̌-m
125         ʔín-jɛ̀
126            úŋɔ̀
127        ùŋɔ́-ŋ́ 
128             ìdú
129             ìzú
130             ìjú
131             ìjú
132           ínjɛ̀
133        ŋ̀g-wɛ̌ɛ
134          nɛ̀rⁿî
135           -ŋkɛ̀
136             ìšú
137          ìnjɛ̌ɛ
138        í-ínjɛ́ 
139        í-í-ŋgɛ́
140             ìsí
141             ìsé
142             ùjù
143            ʔùjù
144            nènú
145           ìnjɛ̀
146            ǹjɛ̀
147             ìjú
148         bɛ́rù-m
149         bɛ́rì-m


In [10]:
use.loc[195]

'bɛ̀r-àjí:'

### Parsing Verbs

In [20]:
verbs = data[data["FORM"].notna()]
verbs=verbs[["VALUE_ORG", "FORM"]]
verbs=verbs["FORM"]

In [21]:
verb_parser=VerbParser()

In [22]:
verbs.head(10).apply(
    lambda x: verb_parser.parse_durationals(x)
).apply(
    lambda x: verb_parser.segment_cvcs(x)
)

424     pánd-í
425         pó
426      pál-í
427      úr-ɔ́
428    ʔɔ́r-ɛ̀
429    kɛ̀s-ɛ́
430      úr-ɛ̀
431      úr-ɛ̀
432     pɔ́-ɔ́
433      pán-á
Name: FORM, dtype: object